In [ ]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
from pycmqlib3.utility import dbaccess, dataseries, misc
from pycmqlib3.analytics.tstool import *
from pycmqlib3.analytics.btmetrics import *
from pycmqlib3.analytics.backtest_utils import *
from pycmqlib3.strategy.signal_repo import *

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
matplotlib.rcParams['figure.figsize'] = (12, 8)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [ ]:
# load historical data

In [ ]:
from misc_scripts.update_fut_prices import load_saved_fut
tday = datetime.date(2025, 5, 16)

df = load_saved_fut(tday, freq='d')
#df = load_cnc_fut(tday, type='cnc')

start_date = df.index[0]
end_date = tday

cdates = pd.date_range(start=start_date, end=tday, freq='D')
bdates = pd.bdate_range(start=start_date, end=end_date, freq='C', holidays=misc.CHN_Holidays)

In [ ]:
from pycmqlib3.utility.misc import day_shift, CHN_Holidays, prod2exch, is_workday, \
    nearby, contract_expiry, inst2contmth

spot_df = load_fun_data(tday)
spot_df['hc_rb_diff'] = np.log(df[('hcc1', 'close')]) - np.log(df[('rbc1', 'close')])    
spot_df['io_ctd_spot'] = io_ctd_basis(spot_df, df[('ic1', 'expiry')])

In [ ]:
spot_dict = {}
spot_dict['cu_base_phybas'] = spot_df['cu_smm_phybasis']
spot_dict['al_base_phybas'] = spot_df['al_smm0_phybasis']
spot_dict['zn_base_phybas'] = spot_df['zn_smm0_sh_phybasis']
spot_dict['pb_base_phybas'] = spot_df['pb_smm1_sh_phybasis']
spot_dict['ni_base_phybas'] = spot_df['ni_smm1_jc_phybasis']
spot_dict['sn_base_phybas'] = spot_df['sn_smm1_sh_phybasis']
spot_dict['ss_base_phybas'] = spot_df["ss_304_wuxi_phybasis"]
spot_dict['cu_tc'] = spot_df['cu_mine_tc']
spot_dict['zn_tc'] = spot_df['zn_50conc_tc_henan']
spot_dict['pb_tc'] = spot_df['pb_50conc_tc_neimeng']
spot_dict['sn_tc'] = spot_df['sn_40conc_tc_yunnan']

metal_inv_map = {
    'cu': "cu_inv_social_dom",
    'al': 'al_inv_social_all',
    'zn': "zn_inv_social_all",
    'ni': "ni_inv27_all",
    'pb': 'pb_inv_social_all',
    'sn': 'sn_inv_social_all',
    'si': 'si_inv_social_all',
    'ao': 'bauxite_inv_az_ports',
    'ss': "ss_inv_social_300",
    'rb': 'rebar_inv_social',
    'hc': 'hrc_inv_social',
    'j': "coke_inv_ports_tj",
    'jm': "ckc_inv_cokery",
    'v': "v_inv_social",
    'i': 'io_inv_45ports',
    'SM': 'sm_inv_mill',
    'SF': 'sf_inv_mill',
    'FG': "fg_inv_mill",
    'SA': 'sa_inv_mill_all',
}
for asset in metal_inv_map:
    spot_dict[f'{asset}_sinv'] = spot_df[metal_inv_map[asset]]
    
spot_dict["zn_scrap_sh_mid"] = (spot_df["zn_scrap_sh_low"] + spot_df["zn_scrap_sh_high"])/2
spot_dict["al_scrap_shredded_sh_mid"] = (spot_df["al_scrap_shredded_sh_low"] + spot_df["al_scrap_shredded_sh_high"])/2
spot_dict['rebar_total_stockdays'] = (spot_df['rebar_inv_social']).dropna()/spot_df['rebar_app_dmd'].dropna().rolling(52).mean()*7
spot_dict['hrc_total_stockdays'] = (spot_df['hrc_inv_social']).dropna()/spot_df['hrc_app_dmd'].dropna().rolling(52).mean()*7    

fef_list = []
for nb in [2, 3, 4]:
    fef_nb = nearby('FEF', n=nb,
                    start_date=max(start_date.date(), datetime.date(2016, 1, 1)),
                    end_date=tday,
                    roll_rule='-3b', freq='d', shift_mode=2, roll_col='settle')
    fef_nb.loc[fef_nb['settle'] <= 0, 'settle'] = np.nan
    fef_nb.loc[fef_nb['close'] <= 0, 'close'] = np.nan
    fef_list.append(fef_nb['settle'].to_frame(f'FEFc{nb-1}'))
    fef_list.append(fef_nb['close'].to_frame(f'FEFc{nb-1}_close'))
    fef_list.append(fef_nb['shift'].to_frame(f'FEFc{nb-1}_shift'))
fef_data = pd.concat(fef_list, axis=1)
fef_data.index = pd.to_datetime(fef_data.index)

for col in fef_data.columns:
    spot_dict[col] = fef_data[col]

spot_dict['hc_rb_diff'] = np.log(df[('hcc1', 'close')]) - np.log(df[('rbc1', 'close')])    
spot_dict['io_ctd_spot'] = io_ctd_basis(spot_df, df[('ic1', 'expiry')])

#product_list = df.
metal_pbc = {
    "cu": "cu_smm1_spot",
    "al": "al_smm0_spot",
    "zn": "zn_smm0_spot",
    "pb": "pb_smm1_spot",
    "sn": "sn_smm1_spot",
    "ni": "ni_smm1_spot",
    "ss": "ss_304_gross_wuxi",
    "ao": "alumina_spot_qd",
    "si": "si_553_spot_smm",
    "rb": "rebar_sh",
    "hc": "hrc_sh",
    "i": "io_ctd_spot",
    "j": "coke_sub_a_rz",
    #"jm": "ckc_a10v24s08_lvliang", #"ckc_a10v24s10_ts", 
    "jm": "ckc_outstock_ganqimaodu", #"ckc_a10v24s08_lvliang",
    "FG": "fg_5mm_shahe",
    #"SM": "sm_65s17_neimeng",
    "SM": "sm_65s17_tj",
    #"SF": "sf_72_ningxia",
    "SF": "sf_72_neimeng",
    
    "v": "pvc_cac2_east",
    #"SA": "sa_heavy_east",
    "SA": "sa_heavy_shahe",
    "au": "au_td_sge",
    "ag": "ag_td_sge",
        
    "MA": "ma_spot_jiangsu",
    "TA": "pta_east_spot",
    "eg": "eg_east_spot",
    "PF": "pf_fujian_spot",
    "l": "l_7042_tj",
    "pp": "pp_100ppi_spot",
    "eb": "eb_east_spot",
    'bu': "bu_heavy_shandong",
    "pg": "pg_jincheng_shandong",
    'ru': "ru_scrwf_kunming",
    'fu': "fo_180cst_xiamen",
}

vol_win = 20
product_list = list(set([col[:-2] for col in df.columns.get_level_values(0).unique()]))

for asset in product_list:    
    spot_dict[f'{asset}_px'] = df[(asset+'c1', 'close')]
    spot_dict[f'{asset}_logret'] = np.log(df[(asset+'c1', 'close')]).dropna().diff()
    spot_dict[f'{asset}_pctchg'] = np.log(df[(asset+'c1', 'close')]).dropna().pct_change()        
    if (asset+'c2', 'close') not in df.columns:
        print(asset)
        continue
    spot_dict[f'{asset}_ryield'] = (np.log(df[(asset+'c1', 'close')]) - np.log(df[(asset+'c2', 'close')]) - 
                                  df[(asset+'c1', 'shift')] + df[(asset+'c2', 'shift')])/(df[(asset+'c2', 'expiry')] - df[(asset+'c1', 'expiry')]).dt.days*365.0 + spot_df['r007_cn'].dropna().ewm(5).mean()/100
    spot_dict[f'{asset}_logret2'] = np.log(df[(asset+'c2', 'close')]).dropna().diff()
    spot_dict[f'{asset}_basmom'] =spot_dict[f'{asset}_logret'] - spot_dict[f'{asset}_logret2']        
    spot_dict[f'{asset}_pctvol'] = spot_dict[f'{asset}_pctchg'].dropna().rolling(vol_win).std()
    spot_dict[f'{asset}_basmom5'] = spot_dict[f'{asset}_basmom'].dropna().rolling(5).sum()
    spot_dict[f'{asset}_basmom10'] = spot_dict[f'{asset}_basmom'].dropna().rolling(10).sum()
    spot_dict[f'{asset}_basmom20'] = spot_dict[f'{asset}_basmom'].dropna().rolling(20).sum()
    spot_dict[f'{asset}_basmom60'] = spot_dict[f'{asset}_basmom'].dropna().rolling(60).sum()
    if asset in metal_pbc:
        asset_feature = metal_pbc[asset]
        spot_dict[f'{asset}_c1'] = df[(asset+'c1', 'close')] / np.exp(df[(asset+'c1', 'shift')])
        spot_dict[f'{asset}_expiry'] = pd.to_datetime(df[(asset+'c1', 'expiry')])
        tmp_df = pd.concat([spot_df[asset_feature].dropna(), 
                            spot_dict[f'{asset}_c1'].dropna().to_frame(f'{asset}_c1'),
                            spot_dict[f'{asset}_expiry'].dropna().to_frame(f'{asset}_expiry'),                            
                            spot_df['r007_cn'].dropna()
                           ], axis=1)
        tmp_df['date'] = pd.to_datetime(tmp_df.index)
        tmp_df['r007_cn'] = tmp_df['r007_cn'].ffill().ewm(5).mean()/100
        #tmp_df['r007_cn'] = tmp_df['shibor_1m'].ffill()/100
        spot_dict[f'{asset}_phycarry'] = (np.log(tmp_df[asset_feature]) - np.log(tmp_df[f'{asset}_c1'])) / \
                                           (tmp_df[f'{asset}_expiry'] - tmp_df['date']).dt.days * 365 + tmp_df['r007_cn']

keys = [key for key in spot_df.columns if key not in spot_dict]
spot_df = pd.concat([spot_df[keys], pd.DataFrame(spot_dict)], axis=1)

In [ ]:
vol_win=20
pnl_tenors = ['6m', '1y', '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y', '10y']
insample = "2026-01-01"

empiric_assets = [
    'ss', 'SA', 'FG', 'l', 'pp', 'v', 'TA', 'MA', 'eg', 'bu', 'fu', 'a', 'c', 'CF',
    #'SM', 'SF',
    "UR", "ru", 'si', 'lc', 'ao', #"PX", 
    #"AP", "CJ",
    #'rb', 'hc', 
    #'jm', 
    #'i',  'j', 
    
    #'cu', 'al', 'zn', 'ni', 'sn', 'pb', 
]

traded_price = 'n305'
df_pxchg = pd.DataFrame(index=df.index)

for asset in empiric_assets:
    if '_' in asset:
        df_pxchg[asset] = beta_ret_dict[asset].dropna()[:insample]
    elif '-' in asset:
        for sub_asset in asset.split('-'):
            if sub_asset not in empiric_assets:
                df_pxchg[sub_asset] = df[(sub_asset+'c1', traded_price)].dropna().pct_change()[:insample]                 
    else:
        if asset in ["T", "TF", "TS", "TL", "IF", "IH", "IC", "IM"]:
            if traded_price in ['n305', 'n310', 'n315', 'n450', 'a1505', 'a1510', 'a1515']:
                ts_px = df[(asset+'c1', 'a1535')]
            else:
                ts_px = df[(asset+'c1', traded_price)]
            flag = ts_px.isna()
            ts_px.loc[flag] = df[(asset+'c1', 'd_twap')].loc[flag]
        else:
            ts_px = df[(asset+'c1', traded_price)]
            if traded_price in ['n305', 'n310', 'n315', 'n450']:
                flag = ts_px.isna()
                ts_px.loc[flag] = df[(asset+'c1', 'a1505')].loc[flag]
            elif traded_price in ['n_twap']:
                flag = ts_px.isna()
                ts_px.loc[flag] = df[(asset+'c1', 'd_twap')].loc[flag]        
                
        df_pxchg[asset] = ts_px.dropna().pct_change()[:insample]        
vol_df = df_pxchg.rolling(vol_win).std()
vol_df2 = pd.DataFrame(index=df.index)
for asset in df_pxchg.columns:
    vol_df2[asset] = robust_vol_calc(df_pxchg[asset])

In [ ]:
cutoff='2010-01-01'
shift_holdings = 1
signal_cap = [-2, 2]
chg_func = 'diff'
bullish = False
vol_win = 52

by_asset = True

signal_func = 'hlratio'
param_rng = [480, 520, 2]
#feature = "sm_margin_north"
#feature = "sf_neimeng_margin"
#feature = "cnh_cny_spd2"
#feature = 'base_sw_csi500_ret'
#feature = 'glass_sw_csi500_ret'
#feature = 'sf_neimeng_cost'
#feature = 'mn_44_gabon_tj'
#feature = "phycarry"
# feature = 'rebar_sales_inv_ratio'
#feature = "pmi_cn_cons_bus_exp"
# feature = 'au_td_sge'
#feature='cnh_cny_spd2'
#feature='etf_holdings'
#feature='mn_44_gabon_tj'
#feature = 'fxbasket_cumret'
#feature="auag_etf_holdings_ratio"
#featrue = "pmi_order_rminv_ratio"
#feature = "auag_cme_warrant_ratio"
#feature = "shibor_1m" #
feature='exch_warrant'

freq=''

signal_df = pd.DataFrame(index=df_pxchg.index)

for asset in empiric_assets:
    #feature_ts = df_pxchg[asset].cumsum()
    #feature_ts = df[(asset+'c1', 'close')].dropna() #.pct_change() 
    if by_asset:
        asset_feature = f"{asset}_{feature}"
    else:
        asset_feature = feature
    feature_ts =spot_df[asset_feature].dropna() #.rolling(100).sum()
    #feature_ts = feature_ts.diff(252)
    #feature_ts = feature_ts.rolling(12).sum()
    #feature_ts.index = feature_ts.index + pd.DateOffset(days=9) + chn_bday * 1
    #feature_ts = np.log(feature_ts)
    #feature_ts = feature_ts.rolling(100).sum()
    #feature_ts = (1+stock_pct_chg[["SPY.P"]].mean(axis=1)).cumprod()
    #feature_ts = (1+stock_pct_chg[['SPY.P']].mean(axis=1)).cumprod()
    #feature_ts = spot_df[asset_feature].ffill().reindex(index=df_pxchg.index)
    #feature_ts = beta_ret_dict[asset].dropna().cumsum()
    #feature_ts = spot_df[feature].ffill().reindex(index=pd.date_range(start=df.index[0], end=df.index[-1], freq=freq)).ffill().dropna()    
    #feature_ts = df[(asset, 'c1', 'close')].dropna()
    #ticker, param_rng = feature_map[asset]    
    #feature_ts = (spot_df[beta_feature_map[asset][0]]/spot_df[beta_feature_map[asset][1]]).dropna()
    #signal_ts = (feature_ts.ewm(8).mean() - feature_ts.ewm(30).mean())/feature_ts.diff().ewm(30).std()
    # feature_ts = beta_residual(
    #     spot_df[feature].dropna(),
    #     df_pxchg[asset].dropna().cumsum(),
    #     beta_win=120, 
    #     chg_func='diff', corr_step=1) 
#     feature_ts = beta_residual(
#         spot_df[beta_feature_map[asset][0]].dropna(),
#         spot_df[beta_feature_map[asset][1]].dropna(),
#         beta_win=250, 
#         chg_func='pct_change') 
    #feature_ts = np.log(feature_ts)
#     if asset in ['cu', 'al']:
#         #param_rng = [1, 2, 1]
#         feature_ts = feature_ts.rolling(20).mean()
#     else:
#         #param_rng = [1, 2, 1]
#         feature_ts = feature_ts.rolling(2).mean()
    feature_ts = feature_ts.reindex(index=cdates).ffill().reindex(index=bdates)
    #feature_ts = feature_ts.ewm(1).mean()
    #feature_ts = yoy_generic(feature_ts, label_func=calendar_label, group_col='label_day', func=chg_func)[asset_feature]    
    #feature_ts = yoy_generic(feature_ts, label_func=lunar_label, group_col='label_day', func=chg_func)[asset_feature]    
    signal_ts = calc_conv_signal(feature_ts, signal_func=signal_func, param_rng=param_rng, signal_cap=signal_cap, vol_win=vol_win) 
    #signal_ts = feature_ts
    #signal_ts = 2*signal_ts - 1*signal_ts.shift(1)
    #signal_ts = np.sign(signal_ts)    
    #signal_ts = conv_ewm(feature_ts, [2, 6, 2], [8, 32, 4], vol_win=60)    
    #signal_ts = zscore_roll(feature_ts, 120)
    #signal_ts = ewmac(feature_ts, 8, 16, vol_win=0)
    #signal_ts = pd.Series(1, index=df_pxchg.index)
    #signal_ts.loc[signal_ts.index.month.isin([1, 2, 3, 4, 5, 6, 7, 12])] = 1
    #signal_ts.loc[signal_ts.index.day.isin(range(16, 32))] += 1
    #signal_ts = zscore_roll(feature_ts, 60)
    #signal_ts = signal_hysteresis(signal_ts, 1.2, 0.6, False)
    #signal_ts = seasonal_score(feature_ts.to_frame(), backward=10, forward=10, rolling_years=5, min_obs=10)
    # signal_ts = seasonal_score(feature_ts.to_frame(), backward=15, forward=15, rolling_years=3, min_obs=30)
    #signal_ts = create_holiday_window_series(df.index, opt_expiries, 1, 3) # - create_holiday_window_series(df.index, opt_expiries, 3, 4)
    if not bullish:
        signal_ts = -signal_ts    
    
    signal_ts = signal_ts.reindex(index=cdates).ffill().reindex(index=df_pxchg.index)
    #signal_ts = signal_ts.rolling(2).mean()
    #signal_ts = signal_ts.ewm(3).mean()
    #signal_ts = signal_ts.shift(1)    
    #signal_ts.loc[signal_ts.index.month.isin([1, 2])] = 0
    #signal_ts = signal_hump(signal_ts, 0.2)
    if '-' in asset:
        sub_assets = asset.split('-')
        if sub_assets[0] in signal_df.columns:
            signal_df[sub_assets[0]] += signal_ts
        else:
            signal_df[sub_assets[0]] = signal_ts
        if sub_assets[1] in signal_df.columns:
            signal_df[sub_assets[1]] -= signal_ts
        else:
            signal_df[sub_assets[1]] = -signal_ts            
    else:
        signal_df[asset] = signal_ts

#signal_df = xs_demean(signal_df)
signal_df = signal_df + xs_demean(signal_df)
#signal_df = signal_buffer(signal_df, 0.05)

holding = generate_holding_from_signal(signal_df, vol_df, risk_scaling=1.0, asset_scaling=False)

bt_metrics = MetricsBase(holdings=holding[signal_df.columns][cutoff:],
                         returns=df_pxchg[signal_df.columns][cutoff:], 
                         shift_holdings=shift_holdings)
trading_cost = dict([(asset, 0.6e-4) if asset in ["T", "TF"] else (asset, 2e-4) for asset in holding.columns])

bt_metrics_w_cost = MetricsBase(holdings=holding[signal_df.columns][cutoff:],
                                returns=df_pxchg[signal_df.columns][cutoff:], 
                                shift_holdings=shift_holdings,
                                cost_dict=trading_cost)

pnl_stats = bt_metrics.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors)
pnl_stats_w_cost = bt_metrics_w_cost.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors)

print("SR after cost:\n", pnl_stats_w_cost['sharpe'])
print(pnl_stats_w_cost['asset_sharpe_stats'])
print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats_w_cost['turnover'], pnl_stats_w_cost['pnl_per_trade']))

print("SR before cost:\n", pnl_stats['sharpe'])
print(pnl_stats['asset_sharpe_stats'])
print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats['turnover'], pnl_stats['pnl_per_trade']))

iplot(pnl_stats_w_cost['portfolio_cumpnl'], title='portfolio pnl with cost')
iplot(pnl_stats_w_cost['asset_cumpnl'], title='asset pnl with cost')

iplot(pnl_stats['portfolio_cumpnl'], title='portfolio pnl wo cost')
iplot(pnl_stats['asset_cumpnl'], title='asset pnl wo cost')


In [ ]:
feature_setup = {
    "exch_wnt_hlr": [
        ['rb', 'hc', "ru", 'si', 'lc', 'ao', 'UR', 'ss', 'SA', 'FG', 'l', 'pp', 'v', 'TA', 'MA', 'eg', 'bu', 'fu', 'a', 'c', 'CF'],
        ["exch_warrant", "hlratio", [230, 250, 2], "", "", False, "", "", 240, [-2,2]]],
    "exch_wnt_hlr_xdemean": [
        ['rb', 'hc', "ru", 'si', 'lc', 'ao', 'UR', 'ss', 'SA', 'FG', 'l', 'pp', 'v', 'TA', 'MA', 'eg', 'bu', 'fu', 'a', 'c', 'CF'],
        ["exch_warrant", "hlratio", [230, 250, 2], "", "", False, "", "", 240, [-2,2]]],    
    "exch_wnt_hlr_mt": [
        ['rb', 'hc', "ru", 'si', 'lc', 'ao', 'UR', 'ss', 'SA', 'FG', 'l', 'pp', 'v', 'TA', 'MA', 'eg', 'bu', 'fu', 'a', 'c', 'CF'],
        ["exch_warrant", "hlratio", [80, 120, 2], "", "", False, "", "ema3", 240, [-2,2]]],
    "exch_wnt_hlr_mt_xdemean": [
        ['rb', 'hc', "ru", 'si', 'lc', 'ao', 'UR', 'ss', 'SA', 'FG', 'l', 'pp', 'v', 'TA', 'MA', 'eg', 'bu', 'fu', 'a', 'c', 'CF'],
        ["exch_warrant", "hlratio", [80, 120, 2], "", "", False, "", "ema3", 240, [-2,2]]],
    "exch_wnt_yoy_hlr": [
        ['rb', 'hc', "ru", 'si', 'lc', 'ao', 'UR', 'ss', 'SA', 'FG', 'l', 'pp', 'v', 'TA', 'MA', 'eg', 'bu', 'fu', 'a', 'c', 'CF'],
        ["exch_warrant", "hlratio", [230, 250, 2], 'cal_yoy_day', "diff", False, "", "ema3", 240, [-2,2]]],
    "exch_wnt_yoy_hlr_xdemean": [
        ['rb', 'hc', "ru", 'si', 'lc', 'ao', 'UR', 'ss', 'SA', 'FG', 'l', 'pp', 'v', 'TA', 'MA', 'eg', 'bu', 'fu', 'a', 'c', 'CF'],
        ["exch_warrant", "hlratio", [230, 250, 2], 'cal_yoy_day', "diff", False, "", "ema3", 240, [-2,2]]],    
}

In [ ]:
cutoff='2010-01-01'
misc_fact = 14000
feature_names = [
    ["exch_wnt_hlr", 0.48*misc_fact],
    ["exch_wnt_hlr_xdemean", 0.36*misc_fact],
    #["exch_wnt_hlr_mt", 1],
    #["exch_wnt_hlr_mt_xdemean", 1],    
    ["exch_wnt_yoy_hlr", 0.12*misc_fact],
    ["exch_wnt_yoy_hlr_xdemean", 0.24*misc_fact], 
]

signal_dict = {}
signal_df = pd.DataFrame(0, columns=empiric_assets, index=cdates)

for feature_name, weight in feature_names:
    signal_assets = feature_setup[feature_name][0]
    feature, signal_func, param_rng, proc_func, chg_func, bullish, freq, post_func, vol_win, signal_cap = feature_setup[feature_name][1] 
    if feature in ['sinv', 'base_phybas', 'prem_bonded_warrant', 'prem_bonded_cif', 
                   'tc', 'phycarry', 'ryield', 'basmom5', 'basmom10', 'basmom20', 'basmom40', 'basmom60', 'basmom120', 
                   'base_inv', 'inv_shfe_d', 'inv_lme_total', 'inv_exch_d', 'px', 'etf_holdings', 'exch_warrant', 'smsf_prodcost']:
        sig_df = pd.DataFrame(0, columns=empiric_assets, index=cdates)
        for asset in empiric_assets:
            if feature == 'base_inv':
                asset_feature = base_inv.get(asset, f"{asset}_{feature}")
            else:
                asset_feature = f"{asset}_{feature}"
            if (asset not in signal_assets) or (asset_feature not in spot_df.columns):
                sig_df[asset] = np.nan
                continue
            if feature in ['base_phybas']:
                if asset in ['cu', 'al']: 
                    proc_func = 'sma20'
                else:
                    proc_func = 'sma2'
            signal_ts = calc_funda_signal(spot_df, asset_feature, signal_func, param_rng,
                                          proc_func=proc_func, chg_func=chg_func, bullish=bullish,
                                          freq=freq, signal_cap=signal_cap, bdates=bdates,
                                          post_func=post_func, vol_win=vol_win)
            sig_df[asset] = signal_ts
        sig_df = sig_df.reindex(index=cdates).ffill().reindex(index=df_pxchg.index)
        if "xdemean" in feature_name:
            sig_df = xs_demean(sig_df)
        elif "xscore" in feature_name:
            sig_df = xs_score(sig_df)
        elif "xrank" in feature_name:
            sig_df = xs_rank(sig_df, 0.2)

    else:
        sig_ts = calc_funda_signal(spot_df, feature, signal_func, param_rng,
                                   proc_func=proc_func, chg_func=chg_func, bullish=bullish,
                                   freq=freq, signal_cap=signal_cap, bdates=bdates,
                                   post_func=post_func, vol_win=vol_win)
        sig_df = pd.DataFrame(0, columns=empiric_assets, index=cdates)
        for asset in signal_assets:
            if '-' in asset:
                sub_assets = asset.split('-')
                if sub_assets[0] in sig_df.columns:
                    sig_df[sub_assets[0]] += sig_ts
                else:
                    sig_df[sub_assets[0]] = sig_ts
                if sub_assets[1] in sig_df.columns:
                    sig_df[sub_assets[1]] -= sig_ts
                else:
                    sig_df[sub_assets[1]] = -sig_ts            
            else:
                sig_df[asset] = sig_ts
        sig_df = sig_df.reindex(index=cdates).ffill().reindex(index=df_pxchg.index)
    last_func = post_func.split('|')[-1]
    if 'buf' in last_func:
        buffer_size = float(last_func[3:])
        sig_df = signal_buffer(sig_df, buffer_size)
    elif 'bfc' in last_func:
        buffer_size = float(last_func[3:])
        sig_df = signal_cost_optim(sig_df, 
                                   buffer_size, 
                                   vol_df, 
                                   cost_dict = dict([(asset, 1e-4) for asset in empiric_assets]))
    sig_df = sig_df.reindex(index=cdates).ffill().reindex(index=df_pxchg.index).fillna(0)
    signal_dict[feature_name] = sig_df
    signal_df = signal_df + signal_dict[feature_name] * weight
    
signal_dict['combo'] = signal_df.dropna().ffill()

In [ ]:
pnl_df = pd.DataFrame(index=df_pxchg.index)
for signal_name in signal_dict:
    signal_df = signal_dict[signal_name]
    holding = generate_holding_from_signal(signal_df, vol_df, risk_scaling=1, asset_scaling=False)    
    
    bt_metrics = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                             returns=df_pxchg[empiric_assets][cutoff:], 
                             shift_holdings=1)

    bt_metrics_w_cost = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                                    returns=df_pxchg[empiric_assets][cutoff:], 
                                    shift_holdings=1,
                                    cost_dict=simple_cost(holding.columns, trd_cost=1e-4))

    pnl_stats = bt_metrics.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors)    
    pnl_stats_w_cost = bt_metrics_w_cost.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors, perf_metrics=['sharpe', 'std', 'sortino', 'calmar'])    
    perf_stats = transform_output(pnl_stats_w_cost)
    pnl_df[signal_name] = pnl_stats_w_cost['portfolio_pnl']['total']
    
    print("signal=%s\n" % signal_name)
    print("SR after cost:\n")
    display(perf_stats)
    print(pnl_stats_w_cost['asset_sharpe_stats'])
    print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats_w_cost['turnover'], pnl_stats_w_cost['pnl_per_trade']))

    print("SR before cost:\n", pnl_stats['sharpe'])
    print(pnl_stats['asset_sharpe_stats'])
    print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats['turnover'], pnl_stats['pnl_per_trade']))

    iplot(pnl_stats_w_cost['portfolio_cumpnl'], title='portfolio pnl with cost')
    iplot(pnl_stats_w_cost['asset_cumpnl'], title='asset pnl with cost')

    iplot(pnl_stats['portfolio_cumpnl'], title='portfolio pnl wo cost')
    iplot(pnl_stats['asset_cumpnl'], title='asset pnl wo cost')

corr_cutoff = '2017-01-01'
print("std:\n%s\n" % pnl_df[corr_cutoff:].std(axis=0))

pnl_w_df = pnl_df.drop(columns=['combo']).resample('W').sum()
print("corr:\n%s\n" % pnl_w_df[corr_cutoff:].corr())

In [ ]:
import pypfopt
from pypfopt import plotting
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import expected_returns

dpnl = pnl_df['2018-01-01':].drop(columns=['combo'])
strat_std = dpnl.std()
dpnl = dpnl/dpnl.std()

max_sharpe = True

if max_sharpe:
    mu = expected_returns.mean_historical_return(dpnl, returns_data=True, frequency=244, compounding=False)
else:
    mu = None

S = risk_models.CovarianceShrinkage(dpnl, returns_data=True, frequency=244).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds= (0, 1)) 

fact_idx = {}
for fact in dpnl.columns:
    fact_idx[fact] = ef.tickers.index(fact)

#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] >= 0.3)
#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] <= 0.4)
#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] >= 0.3)
# ef.add_constraint(lambda w: w[fact_idx['tsmom']] + w[fact_idx['macro2']] <= 0.24)

#ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
if max_sharpe:
    port_weights = ef.max_sharpe()
else:
    port_weights = ef.min_volatility()

#port_weights = ef.clean_weights()
port_weights = pd.Series(port_weights)
print("port_weights=\n%s\n" % port_weights)
final_weights = port_weights.div(strat_std)
print("final_weights=\n%s\n" % final_weights)
# mu = expected_returns.mean_historical_return(df)
# S = risk_models.sample_cov(df)

# # Optimize for maximal Sharpe ratio
# ef = EfficientFrontier(mu, S)
# weights = ef.max_sharpe()
# ef.portfolio_performance(verbose=True)

In [ ]:
lead_lag_config = {
    'll_left': -20,
    'll_right': 120,
    'll_spacing': 5,
    'll_sub_win': [(datetime.date(2008, 1, 1), datetime.date(2018, 12, 31)), 
                   (datetime.date(2019, 1, 1), datetime.date(2024, 12, 31)),],
}

ll_keys = ['fullsample'] + ['%s:%s' % (sd.strftime('%Y-%b-%d'), ed.strftime('%Y-%b-%d')) for sd, ed in lead_lag_config['ll_sub_win']]


ll_left = lead_lag_config['ll_left']
ll_right = lead_lag_config['ll_right']
spacing = lead_lag_config['ll_spacing']

leadlag_df = bt_metrics.lead_lag(ll_limit_left=ll_left, 
                                 ll_limit_right=ll_right,
                                 ll_sub_windows=lead_lag_config['ll_sub_win'])

fig, ax = plt.subplots(len(ll_keys), 1)
fig.set_figheight(15)
fig.set_figwidth(10)

for i, key in enumerate(ll_keys):
    ts = leadlag_df['leadlag_sharpes'].loc[key]
    ts.plot(kind='bar', ax = ax[i], title = f'lead_lag: {key}')
    new_ticks = np.linspace(ll_left, ll_right, (ll_right-ll_left)//spacing+1)
    ax[i].set_xticks(np.interp(new_ticks, ts.index, np.arange(ts.size)))
    ax[i].set_xticklabels(new_ticks)
    ax[i].axvline(x=-ll_left, color='red', linestyle='--')
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111)
ls_pnl = bt_metrics.long_short_pnl()
for key in ls_pnl:
    ax.plot(ls_pnl[key]['portfolio_cumpnl'], '-', label=key)
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, bbox_to_anchor=(1.04, 1), loc='upper left')
ax.grid()
plt.title("long-short pnl")
plt.show()

lagged = bt_metrics.lagged_pnl(lags=[1, 5, 10, 20, 30, 60, 75, 80])
lagged['cumpnl'].plot()
#print('lagged PNL\n', lagged['sharpe'])
plt.grid()
plt.title('lagged pnl')
plt.show()

smoothed = bt_metrics.smoothed_pnl(smooth_hls=[1, 5, 10, 20, 30, 60, 75, 80])
smoothed['cumpnl'].plot(figsize=(8, 6))
#print('smoothed PNL\n', smoothed['sharpe'])
plt.grid()
plt.title('smoothed pnl')
plt.show()

#tilt_timing = bt_metrics.tilt_timing(tilt_rolling_window=1*244) # default 3 years  tilt_rolling_window = 3 * 244 

seasonal_pnl = bt_metrics.seasonal_pnl()
cumpnl = seasonal_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('seasonal sharpe stats\n', seasonal_pnl['sharpe_stats'])
plt.grid()
plt.title('monthly pnl')
plt.show()


monthday_pnl = bt_metrics.monthday_pnl()
cumpnl = monthday_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('monthday sharpe stats\n', monthday_pnl['sharpe_stats'])
plt.grid()
plt.title('monthday pnl')
plt.show()


week_pnl = bt_metrics.week_pnl()
cumpnl = week_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('week sharpe stats\n', week_pnl['sharpe_stats'])
plt.grid()
plt.title('weekday pnl')
plt.show()


annual_pnl = bt_metrics.annual_pnl()
cumpnl = annual_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('annual sharpe stats\n', annual_pnl['sharpe_stats'])
plt.grid()
plt.title('annual pnl')
plt.show()

annual_pnl['cumlog_pnl'].mean(axis=1).plot()
plt.grid()
plt.title('annual averaged profile')
plt.show()

# turnover = bt_metrics.turnover()
# print(turnover)

In [ ]:
# batch feature exploration

In [ ]:
feature_list = [
#     'margin_hrc_sh', 
    'strip_hsec',
    'strip_3.0x685',
    'pipe_1.5x3.25',    
    'hrc_sh',
    'crc_sh',
    'billet_ts',
    'macf_cfd',
#     'gi_0.5_sh',
#     'hsec_400x200',
#     'highwire_6.5',
#     'angle_50x5',
#     'ibeam_25',
#     'channel_16',

#     'import_arb', 'pbf_prem', 'plt65_62',
#     'io_laytime_45ports', 'io_inv_imp_31ports',
#     'io_invdays_imp_mill(64)', 'io_inv_mill(64)', 'io_inv_imp_mill',
#     'io_removal_port_41',
#     'io_loading_14ports_ausbzl',
]

udf = spot_df[feature_list].dropna(how='all')
# lunar_seasonal = True

# if lunar_seasonal:
#     seasonal_signal = tstool.lunar_label(udf)
#     seasonal_signal = tstool.seasonal_group_score(
#         seasonal_signal, score_cols=feature_list, yr_col='lunar_cny',
#         group_col='lunar_wks', min_obs=3, backward=2, forward=2, rolling_years=3)
#     seasonal_signal = seasonal_signal.reindex(index=df.index).ffill()

for feature in udf.columns:
    dataseries.plot_seasonal_df(udf[feature].dropna(), cutoff='2018-01-01', title=feature)
    
signal_raw = udf[feature_list].reindex(index=df.index).ffill()


In [ ]:
cutoff = '2012-01-01'
signal_func = 'qtl'
param_rng = [20, 42, 2]
signal_cap = None # [-2, 2]
product_list = ['rb', 'hc', 'i', 'j', 'jm', 'SF', 'FG', ] # 'v', 'cu', 'al', 'ss', 'UR', 'SA', 'ru'

for asset in product_list:
    if '_' in asset:
        price_ts = (1 + beta_ret_dict[asset]).cumprod().to_frame('price')[cutoff:]
    else:
        price_ts = df[(asset, 'c1', 'close')].dropna().to_frame('price')[cutoff:]
    pnl_list = [price_ts]
    for feature in feature_list:
        feature_ts = udf[feature].reindex(index=price_ts.index).ffill()
        #feature_ts = feature_ts.pct_change(5)
        #feature_ts = tstool.lunar_yoy(feature_ts, group_col='lunar_days', func='pct_change')
        #feature_ts = tstool.seasonal_score(feature_ts.to_frame())
        signal_ts = calc_conv_signal(feature_ts, signal_func=signal_func, param_rng=param_rng, signal_cap=signal_cap)
        asset_df = pd.concat([price_ts, signal_ts], axis=1)
        asset_df.columns = ['price', 'signal']
        asset_df['signal'] = asset_df['signal'].apply(lambda x: x).ffill()
        asset_df = asset_df.dropna(subset=['price'])
        asset_df['position'] = (asset_df['signal']/asset_df['price'].pct_change().rolling(20).std()).shift(1).fillna(0)
        asset_df['pnl'] = (asset_df['position'].shift(1) * asset_df['price'].pct_change()).fillna(0)
        
        sr = np.sqrt(244) * asset_df['pnl'].mean()/asset_df['pnl'].std()
        pnl_per_trade = 100 * 100 * asset_df['pnl'].mean()/asset_df['position'].diff().abs().mean()
        turnover = 100 * asset_df['position'].diff().abs().mean()/asset_df['position'].abs().mean()
        print(f'{asset}:{feature} -> SR: {sr:.2f} -- PNL per trade: {pnl_per_trade:.2f} -- Turnover: {turnover:.2f}')
        pnl_list.append(asset_df['pnl'].cumsum().to_frame(feature))
    pnl_df = pd.concat(pnl_list, axis=1)
    dataseries.plot_df_on_2ax(pnl_df, left_on=feature_list, right_on=['price'])
    

In [ ]:
# signal grid search run

In [ ]:
fill_backward = False
smooth_win = 1
sig_smooth = tstool.exp_smooth(df_in, hl = smooth_win, fill_backward=fill_backward)

demean = False
mean_win = 244
vol_win = 244
if demean:
    sig_scored = tstool.ts_score(sig_smooth, hl_mean=mean_win, min_obs_mean=mean_win, fill_backward_mean=fill_backward, 
                          hl_vol=vol_win, min_obs_vol=vol_win, fill_backward_vol=fill_backward)
else:
    sig_scored = tstool.ts_scale(sig_smooth, hl = vol_win, min_obs=vol_win, fill_backward=fill_backward)

#sig_scored = tstool.xs_score(sig_smooth, demean=demean, hl=vol_win)

signal_cap = 2.0

score_capped = tstool.cap(sig_scored, -signal_cap, signal_cap)
score_filled = tstool.filldown(score_capped, 2)
score = tstool.lag(score_filled, 1)


In [ ]:
vol_scale = 20
asset_vol = tstool.exp_smooth(df_pxchg**2, hl=vol_scale, fill_backward=fill_backward)**0.5
holding = score/asset_vol

commod_list = holding.columns #['hc']
btmetrics = MetricsBase(holdings = holding[commod_list], returns = df_pxchg[commod_list])